In [4]:
### Different approach - batch size learning 
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
import random

import sys
sys.path.append("..")
import fede.utils
from fede.fedavg import Fedavg
from fede.fed_transfer import Fed_Avg_Client
from fede.supported_modles import Supported_modles

In [5]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [6]:
fedavg = Fedavg("global")
lr = 0.05
fedavg.init_global_model(lr, Supported_modles.SGD_classifier, None,78)
applicable_clients = fedavg.wait_for_data(5)

Server is Listening.....


In [7]:
applicable_clients.name

'client1'

In [4]:
selected_model = Supported_modles.SGD_classifier
number_of_rounds = 2
batch_size = 0.05
epochs = 20
# prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    #applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0

    applicable_clients = fedavg.wait_for_data(5)
    
    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X_train, X_test, y_train, y_test = train_test_split(client.x, client.y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
        # X_train = prep.fit_transform(X_train)
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)
        
        #send datasize and sample weights and X_train and y_train and client.model by client

        fedavg.load_global_model(client.model, selected_model) #load global model on the client model

        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model) #make partial fit on globsl model

        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
Server is Listening.....
Connected by ('127.0.0.1', 45132)


OSError: [Errno 107] Transport endpoint is not connected